Nenwin-project (NEural Networks WIthout Neurons) for
the AI Honors Academy track 2020-2021 at the TU Eindhoven.

Authors: Lulof Pirée

May 2021

Copyright (C) 2021 Lulof Pirée

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU Affero General Public License as published
by the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU Affero General Public License for more details.

You should have received a copy of the GNU Affero General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

# Banknote dataset with Nenwin

Dataset source: https://code.datasciencedojo.com/datasciencedojo/datasets/blob/master/Banknote%20Authentication/data_banknote_authentication.txt
See also:
* https://jamesmccaffrey.wordpress.com/2020/08/18/in-the-banknote-authentication-dataset-class-0-is-genuine-authentic/
* https://www.researchgate.net/publication/266673146_Banknote_Authentication

This dataset has:
* 1372 samples, of two classes:
    * class 0 (Genuine): 762 samples 
    * class 1 (Forgery): 610 samples

Features:
* 0: variance (float)
* 1: skewness (float)
* 2: curtosis (float)
* 3: entropy  (float)


**Goal**: classify which banknotes are real and which ones are forgeries,

based on the four features (that are derived from an image, using the 'Wavelet Transform").

In [1]:
import torch
import torch.nn
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Iterable
from numbers import Number

from nenwin.constants import BANKNOTE_CHECKPOINT_DIR
from nenwin.all_particles import MarbleEaterNode, Marble, Node, MarbleEmitterNode
from nenwin.model import NenwinModel
from nenwin.input_placer import InputPlacer
from nenwin.grid_input_placer import VelInputPlacer
from nenwin.attraction_functions.attraction_functions import NewtonianGravity, AttractionFunction
from nenwin.backprop.filename_gen import FilenameGenerator
from nenwin.backprop.trainer import NenwinTrainer
from nenwin.backprop.training_stats import TrainingStats
from nenwin.backprop.loss_function import NenwinLossFunction
from nenwin.banknote_dataset.load_dataset import load_banknote_dataset
from nenwin.auxliary import gen_nodes, gen_eater_nodes

In [2]:
def gen_architecture() -> Tuple[NenwinModel, VelInputPlacer, Tuple[Node]]:
    """
    Generate the following architecture:
    * The input region is at (-2.5, -1) and has size (5, 2) 
        (So it has vertices {(-2.5, -1), (-2.5, 1), (2.5, -1), (2.5, 1)})
    * There are two MarbleEaterNodes, at (-10, 0) and (10, 0)
    * There are four normal Nodes, at (0, -5), (-5, 0), (5, 0) and (0, 5).

    Returns:
    * Model holding the architecture descibed above
    * VelInputPlacer with the input region as described above.
    * Tuple of the two MarbleEaterNodes
    """

    eater_positions = [(-10, 0), (10, 0)]
    node_positions = [(0, -5), (-5, 0), (5, 0), (0, 5)]
    input_region_pos = (-2.5, -1)
    input_region_size = (5, 2)
    mass = 1
    radius = 0.5

    attraction_function = NewtonianGravity()

    nodes = gen_nodes(attraction_function, mass, node_positions)
    eater_nodes = gen_eater_nodes(attraction_function, mass, 
                                  radius, eater_positions)
    model = NenwinModel(nodes+eater_nodes)
    input_placer = VelInputPlacer(input_region_pos, input_region_size)

    return model, input_placer, eater_nodes

In [3]:
gen_architecture()

(NenwinModel({MarbleEaterNode(tensor([10.,  0.]),tensor([0., 0.]),tensor([0., 0.]),1.0,NewtonianGravity(),1.0,1.0,1.0,0.0,0.5), Node(tensor([-5.,  0.]),tensor([0., 0.]),tensor([0., 0.]),1.0,NewtonianGravity(),1.0,1.0,1.0,0.0), Node(tensor([5., 0.]),tensor([0., 0.]),tensor([0., 0.]),1.0,NewtonianGravity(),1.0,1.0,1.0,0.0), Node(tensor([0., 5.]),tensor([0., 0.]),tensor([0., 0.]),1.0,NewtonianGravity(),1.0,1.0,1.0,0.0), MarbleEaterNode(tensor([-10.,   0.]),tensor([0., 0.]),tensor([0., 0.]),1.0,NewtonianGravity(),1.0,1.0,1.0,0.0,0.5), Node(tensor([ 0., -5.]),tensor([0., 0.]),tensor([0., 0.]),1.0,NewtonianGravity(),1.0,1.0,1.0,0.0)},set()),
 [MarbleEaterNode(tensor([-10.,   0.]),tensor([0., 0.]),tensor([0., 0.]),1.0,NewtonianGravity(),1.0,1.0,1.0,0.0,0.5),
  MarbleEaterNode(tensor([10.,  0.]),tensor([0., 0.]),tensor([0., 0.]),1.0,NewtonianGravity(),1.0,1.0,1.0,0.0,0.5)])